In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Aplicaciones Avanzadas/DataBank"
!ls

/content/drive/MyDrive/Aplicaciones Avanzadas/DataBank
fake_news_dataset.csv


In [31]:
# Importación de librerías
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

# Obtener el set de datos

Se obtuvo el dataset de noticias falsas de la plataforma Kaggle y se almacenó en Google Drive

In [14]:
# Lectura del archivo
df = pd.read_csv('fake_news_dataset.csv')

In [15]:
# Vista inicial del dataset
print(df.head())
print(df.info())
print(df.isnull().sum())

                                  title  \
0               Foreign Democrat final.   
1   To offer down resource great point.   
2          Himself church myself carry.   
3                  You unit its should.   
4  Billion believe employee summer how.   

                                                text        date    source  \
0  more tax development both store agreement lawy...  2023-03-10  NY Times   
1  probably guess western behind likely next inve...  2022-05-25  Fox News   
2  them identify forward present success risk sev...  2022-09-01       CNN   
3  phone which item yard Republican safe where po...  2023-02-07   Reuters   
4  wonder myself fact difficult course forget exa...  2023-04-03       CNN   

                 author    category label  
0          Paula George    Politics  real  
1           Joseph Hill    Politics  fake  
2        Julia Robinson    Business  fake  
3  Mr. David Foster DDS     Science  fake  
4         Austin Walker  Technology  fake  
<class '

In [16]:
# Dimension del dataset (filas, columnas)
df.shape

(20000, 7)

# Separación del set de pruebas y del set de entrenamiento

Dividimos el dataset para tener el conjunto que será para entrenamiento y el conjunto que será para pruebas (train y test)

***Label Encoder***

Para distinguir el target a binario,  se utilizará el Label encoder, ya que:

*   Solo hay dos categorias (fake, real)
*   Normaliza etiquetas en valores numéricos
*   Se utiliza menos memoria


In [21]:
target = df["label"]
le = LabelEncoder()
train_label_data = le.fit_transform(target)

In [24]:
# Rellenar valores nulos
df = df.fillna('')

# Combinación del título y texto de la noticia
df['content'] = df['title'] + ' ' + df['text']

# Features y etiquetas
X = df['content']
y = df['label']

In [25]:
# 80% train y 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocesado de datos

### Vectorización del texto (TF-IDF)

In [33]:
# Vectorización de los sets de prueba y de entrenamiento
vectorizer = TfidfVectorizer(max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


El **StandardScaler** se utiliza para estandarizar los datos, es decir, para escalar las características de los datos de entrada de modo que tengan una media de 0 y una desviación estándar de 1.

Como es una **matriz dispersa** en la que la mayoría de los valores son ceros (como las que suelen generarse al usar representaciones como TF-IDF en procesamiento de texto), se desactiva esta opción. Solo se toma en cuenta la desviación estandar. Se debe tener cuidado al escalar los datos para no romper su estructura dispersa.

In [32]:
scaler = StandardScaler(with_mean=False)  # with_mean=False porque es sparse matrix
X_train_scaled = scaler.fit_transform(X_train_tfidf)
X_test_scaled = scaler.transform(X_test_tfidf)
